In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import random
import os
from torch import optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms
import torch.nn as nn
from torchvision.utils import make_grid
from torchvision.utils import save_image
from IPython.display import Image
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import shutil,os,os.path
from os import listdir
from os.path import isfile, join
from PIL import Image, ImageEnhance
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import os, subprocess, zipfile
from tqdm.autonotebook import tqdm

In [ ]:
!pip install tqdm lap
!pip install efficientnet_pytorch

In [ ]:
# Connet to Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

# GPU/CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
SEED = 1000

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
def load_dataset_helper(download_dir, working_dir, datafile):
  if not os.path.exists(os.path.join(working_dir,datafile)):
        base_dir = '/content/work/'

        if os.path.isfile(os.path.join(base_dir,datafile)) == False: 
          !rsync -ah --progress '$download_dir'$datafile '$base_dir'
        else:
          print('already exists!')

        try:
          os.mkdir(working_dir)
        except OSError:
          print ("Creation of the directory %s failed" % working_dir)
        else:
          print ("Successfully created the directory %s " % working_dir)

        #with zipfile.ZipFile(os.path.join(base_dir,datafile)) as zf:
        #  for member in tqdm(zf.infolist(), desc='Extracting '):
        #      try:
        #        zf.extract(member, working_dir)
        #      except zipfile.error as e:
        #          pass
        path_to_zip = os.path.join(download_dir,datafile)
        print(path_to_zip)
        !unzip $path_to_zip -d $working_dir

        print("Directory " , working_dir ,  " Created ")
  else:    
        print("Directory " , working_dir ,  " already exists")


def load_dataset(first_datafile = 'coins_kings'):
    first_datafile += ".zip"
    #YOU NEED TO CHANGE THIS DOWNLOAD_DIR PATH FOR YOUR VALID PATH
    download_dir = '/content/gdrive/My\ Drive/magisterka/'
    working_dir = '/content/work'
    #working_dir_task_2_3 = '/content/work/task_2_3'

    load_dataset_helper(download_dir, working_dir, first_datafile )
    #load_dataset_helper(download_dir, working_dir_task_2_3, second_datafile )


In [ ]:
def give_mean_std(datasetloader):
  pop_mean = []
  pop_std0 = []
  pop_std1 = []

  images, labels = iter(datasetloader).next()

  numpy_image = images.numpy()
  
  batch_mean = np.mean(numpy_image, axis=(0,2,3))
  batch_std0 = np.std(numpy_image, axis=(0,2,3))
  batch_std1 = np.std(numpy_image, axis=(0,2,3), ddof=1)
  
  pop_mean.append(batch_mean)
  pop_std0.append(batch_std0)
  pop_std1.append(batch_std1)

  pop_mean = np.array(pop_mean).mean(axis=0)
  pop_std0 = np.array(pop_std0).mean(axis=0)
  pop_std1 = np.array(pop_std1).mean(axis=0)
  print(str(pop_mean) + " " + str(pop_std0) + " " + str(pop_std1))

In [ ]:
!rm -R /content/work
file_type = 'coins_kings'
for i in range(5):
  file = 'folds_data_packages/' + file_type +"_fold"+str(i+1)
  load_dataset(file)

In [ ]:
class OneCoinSideDataset(torchvision.datasets.ImageFolder):
  def __init__(self, root, kings, transform):
    super(OneCoinSideDataset, self).__init__(root, transform)
    self.kings = kings

  def __getitem__(self, index):
    """
    Args:
      index (int): Index
    Returns:
      tuple: (sample, resnet, target) where target is class_index of the target class.
    """
    path, target = self.samples[index]
    sample = self.loader(path)
    if self.kings:
      sample = sample.crop((0, 0, sample.width/2, sample.height))
    else:
      sample = sample.crop(((sample.width/2), 0, sample.width, sample.height))
    if self.transform is not None:
      sample = self.transform(sample)
    if self.target_transform is not None:
      target = self.target_transform(target)
    return sample, target

In [ ]:
folds_paths=[]
for i in range(5):
  folds_paths.append('/content/work/'+file_type+"_fold"+str(i+1))

dataset_size = 10000
TRAIN_PCT = 0.81
trainset_size = int(dataset_size * TRAIN_PCT)
valset_size = dataset_size - trainset_size
width = 260
height = 260
dimensions = (width, height)

#400 x 500
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader, random_split

#Applying Transformation
means,stds = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
train_transforms = transforms.Compose([
                                transforms.Resize(dimensions),
                                transforms.RandomRotation(5),
                                #transforms.RandomResizedCrop(224),
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(),
                                transforms.Normalize(means,stds),
                                ])

means,stds = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
test_transforms = transforms.Compose([transforms.Resize(dimensions),
                                      #transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize(means,stds),
                                      ])

batch_size=32
test_set_fold_indx = 4 #0-4

#test_data = OneCoinSideDataset(folds_paths[test_set_fold_indx],False, transform=test_transforms)
#del folds_paths[test_set_fold_indx]
#for i in range(len(folds_paths)):
#  folds_paths[i] = OneCoinSideDataset(folds_paths[i],False, transform=train_transforms) 

test_data = torchvision.datasets.ImageFolder(folds_paths[test_set_fold_indx], transform=test_transforms)
del folds_paths[test_set_fold_indx]
for i in range(len(folds_paths)):
  folds_paths[i] = torchvision.datasets.ImageFolder(folds_paths[i], transform=train_transforms)   

train_data = torch.utils.data.ConcatDataset(folds_paths) 

#train_data = OneCoinSideDataset(train_dir,True, transform=train_transforms)                                       
#test_data = OneCoinSideDataset(test_dir,True, transform=test_transforms)
#train_data, val_data = random_split(train_data, [trainset_size, valset_size])

trainloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True)

give_mean_std(trainloader)
give_mean_std(testloader)


In [ ]:
  for img in train_data.imgs:
    width, height = Image.open(img[0]).size
    widthh += width
    heighth += height

  for img in test_data.imgs:
    width, height = Image.open(img[0]).size
    widthh += width
    heighth += height


  print(widthh/(len(train_data.imgs) + len(test_data.imgs)))
  print(heighth/(len(train_data.imgs) + len(test_data.imgs)))
 

In [ ]:
def plot_images2(loader, n_batches):

    rows = int(np.sqrt(n_batches*batch_size))+1
    cols = int(np.sqrt(n_batches*batch_size))+1
    fig = plt.figure(figsize = (8, 8))
    loader_iter = iter(loader)
    for i in range(n_batches):
        images, labels = next(loader_iter)
        images = images.permute(0,2,3,1)
        for j in range(batch_size):
          ax = fig.add_subplot(rows, cols, i*batch_size+j+1)
          ax.imshow(images[j])
          ax.set_title(int(labels[j]))
          ax.axis('off')
    #print(loader.dataset.class_to_idx)

plot_images2(testloader,1)
plot_images2(trainloader,1)

In [ ]:
from torchvision import models
from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_pretrained('efficientnet-b2', num_classes=30).to(device)
#resnet = EfficientNet.from_name('efficientnet-b0', num_classes=30).to(device)

#resnet = models.resnet34(pretrained=False, num_classes = 2).to(device)

#model = models.resnet34(pretrained=True).to(device)
#num_ftrs = resnet.fc.in_features
#resnet.fc = nn.Linear(num_ftrs, 2).to(device)

save_path = '/content/gdrive/MyDrive/magisterka/models/FOLD4_KINGS_BOTHSIDEACCE.pt'
torch.cuda.empty_cache()
optimizer = optim.Adam(model.parameters(), lr = 0.01)
num_epochs = 100
criterion = nn.CrossEntropyLoss()

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import sklearn.metrics as metrics

def save_checkpoint(save_path, model, optimizer, val_loss):
    if save_path==None:
        return
    save_path = save_path 
    state_dict = {'model_state_dict': model.state_dict(),
                  'optimizer_state_dict': optimizer.state_dict(),
                  'val_loss': val_loss}

    torch.save(state_dict, save_path)

    print(f'Model saved to ==> {save_path}')

def load_checkpoint(model, optimizer, path):
    save_path = path
    state_dict = torch.load(save_path, map_location=torch.device(device))
    model.load_state_dict(state_dict['model_state_dict'])
    optimizer.load_state_dict(state_dict['optimizer_state_dict'])
    val_loss = state_dict['val_loss']
    print(f'Model loaded from <== {save_path}')
    
    return val_loss


def train(model, train_loader, test_loader, num_epochs, criterion, save_name):
    best_val_loss = float("Inf") 
    train_losses = []
    val_losses = []
    cur_step = 0
    best_accuracy = 5
    for epoch in range(num_epochs):
        running_loss = 0.0
        accuracy = eval(model, test_loader)
        if best_accuracy < accuracy:
            best_accuracy = accuracy
            save_checkpoint(save_name, model, optimizer, train_losses)
        
        model.train()
        print(" Starting epoch " + str(epoch+1))
        for img1, labels in train_loader:
            # Forward
            img1 = img1.to(device)
            labels = labels.to(device)
            outputs = model(img1)
            loss = criterion(outputs, labels)
            
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)
        train_losses.append(avg_train_loss)
        print("Training loss " + str(train_losses[-1]))
        
    print("Finished Training")  
    return train_losses, val_losses  

# evaluation metrics
def eval(model, test_loader):
    y_test = []
    y_pred = []
    with torch.no_grad():
        model.eval()
        correct = 0
        count = 0
        for img1, labels in test_loader:
            # Forward
            img1 = img1.to(device)
            outputs = model(img1).to(device)
            _, preds = torch.max(outputs, 1)
            y_pred.extend(preds.tolist())
            y_test.extend(labels.tolist())
            labels = labels.to(device)
            for i in range(len(labels)):
              if labels[i] == preds[i]:
                correct = correct+1
              count = count +1

        accuracy = (correct/count)*100
    print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
    if (False):
      print("Precision:",metrics.precision_score(y_test, y_pred, average='macro'))    
      print("F1 score:",metrics.f1_score(y_test, y_pred, average='macro'))
      cm = metrics.confusion_matrix(y_test, y_pred)
      print("Recall:",metrics.recall_score(y_test, y_pred, average='macro'))
      print("Confusion matrix:",cm)
      cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
      print(cm.diagonal())
      #return accuracy
    return accuracy

In [ ]:
num_epochs = 100
train_losses, val_losses = train(model, trainloader, testloader, num_epochs, criterion, save_path)

In [ ]:
print(len(train_losses))
train_losses2 = train_losses


In [ ]:
train_losses = train_losses2 + train_losses

In [ ]:
save_path ='/content/gdrive/MyDrive/magisterka/models/' + 'FOLD5_KINGS_BOTHSIDEACC85.16E82.pt'
#t_loader = trainloader
t_loader = testloader
train_losses = load_checkpoint(model, optimizer, save_path)
#accuracy = eval (model,t_loader)

In [ ]:
img = next(iter(testloader))
print(img[1])
img = img[0]

In [ ]:
named_layers = dict(model.named_modules())
for moudle in model.named_modules():
  print(moudle)

In [ ]:
print(named_layers['_blocks.22._project_conv'])

In [ ]:
!pip install ttach
!pip install grad-cam


from pytorch_grad_cam import GradCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from torchvision.models import resnet50
import argparse

torch.cuda.empty_cache()
target_layer = named_layers['_conv_head']
input_tensor = img# Create an input tensor image for your model..
# Note: input_tensor can be a batch tensor with several images!

# Construct the CAM object once, and then re-use it on many images:
cam = GradCAM(model=model, target_layer=target_layer, use_cuda=False)

# If target_category is None, the highest scoring category
# will be used for every image in the batch.
# target_category can also be an integer, or a list of different integers
# for every image in the batch.
target_category = 29

# You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
grayscale_cam = cam(input_tensor=input_tensor, target_category=target_category)

In [ ]:
index = 22

MEAN = torch.tensor([0.485, 0.456, 0.406])
STD = torch.tensor([0.229, 0.224, 0.225])

x = img[index] * STD[:, None, None] + MEAN[:, None, None]

grayscale_c = grayscale_cam[index, :]
rgb_img = np.float32(x.permute(1,2,0))
visualization = show_cam_on_image(rgb_img, grayscale_c, use_rgb = True)
#plt.imshow(visualization, cmap='gray')
plt.figure()

f, axarr = plt.subplots(1,2) ;



# use the created array to output your multiple images. In this case I have stacked 4 images vertically
axarr[0].imshow(visualization)
axarr[1].imshow(x.permute(1,2,0))

In [ ]:
print(train_losses)
#plotting of training and validation loss
plt.xlabel('epoch')
plt.ylabel('loss')
plt.plot(train_losses, label='Train Loss')
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.show()